<a href="https://colab.research.google.com/github/Kekc/dogsML/blob/main/dogsml/colab/dog_breeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar

--2022-06-08 17:12:32--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘images.tar’

images.tar          100%[===================>] 756.82M  38.9MB/s    in 21s     

2022-06-08 17:12:53 (36.6 MB/s) - ‘images.tar’ saved [793579520/793579520]



In [ ]:
import tarfile
tar_file = tarfile.open("./images.tar", 'r')
tar_file.extractall("tmp/")
tar_file.close()



In [ ]:
import os
images_path = "tmp/Images"

for item in os.listdir(images_path):
  os.rename(os.path.join(images_path, item), os.path.join(images_path, item[item.find("-")+1:]))


In [ ]:
import os
os.listdir("tmp/Images")

['Pembroke',
 'English_springer',
 'Norwich_terrier',
 'Lakeland_terrier',
 'kelpie',
 'Boston_bull',
 'Rottweiler',
 'Great_Dane',
 'bloodhound',
 'Scotch_terrier',
 'bull_mastiff',
 'miniature_poodle',
 'Brabancon_griffon',
 'briard',
 'Tibetan_mastiff',
 'papillon',
 'basenji',
 'Saint_Bernard',
 'Maltese_dog',
 'EntleBucher',
 'Bedlington_terrier',
 'Japanese_spaniel',
 'groenendael',
 'Irish_setter',
 'dhole',
 'Gordon_setter',
 'otterhound',
 'cocker_spaniel',
 'komondor',
 'standard_schnauzer',
 'clumber',
 'Bouvier_des_Flandres',
 'Sealyham_terrier',
 'Border_terrier',
 'miniature_pinscher',
 'silky_terrier',
 'Australian_terrier',
 'Siberian_husky',
 'miniature_schnauzer',
 'Walker_hound',
 'affenpinscher',
 'bluetick',
 'Norwegian_elkhound',
 'dingo',
 'Norfolk_terrier',
 'boxer',
 'Cardigan',
 'basset',
 'German_shepherd',
 'West_Highland_white_terrier',
 'cairn',
 'redbone',
 'Staffordshire_bullterrier',
 'giant_schnauzer',
 'pug',
 'Welsh_springer_spaniel',
 'vizsla',
 'Es

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.optimizers import Adam

In [ ]:
img_height = 224
img_width = 224
batch_size = 32
validation_split = 0.2
seed = 123
num_epochs = 1
data_dir = "tmp/Images"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)
validation_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [ ]:
class_names = train_ds.class_names

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
input_shape = (img_height, img_width, 3)
base_model = tf.keras.applications.MobileNetV3Large(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet",
)

data_augmentation = tf.keras.Sequential([
    tfl.RandomFlip("horizontal"),
    tfl.RandomRotation(0.2),
    # tfl.RandomContrast(factor=0.2),
    # tfl.RandomHeight(factor=0.3),
    # tfl.RandomWidth(factor=0.3),
    # tfl.RandomTranslation(
    #     height_factor=0.3,
    #     width_factor=0.3,
    #     fill_mode="nearest",
    # ),
    tfl.RandomZoom(
        height_factor=(0, 0.3),
        width_factor=(0, 0.3),
        fill_mode="nearest",
    ),
])

base_model.trainable = False
inputs = tf.keras.Input(shape=input_shape)

x_data = data_augmentation(inputs)
# x_data = inputs
# x_data = tf.keras.applications.mobilenet_v2.preprocess_input(x_data)

x_data = base_model(x_data, training=False)
x_data = tfl.GlobalAveragePooling2D()(x_data)
# x_data = tfl.Dropout(0.2)(x_data)
outputs = tfl.Dense(
    len(class_names),
    activation="softmax",
    kernel_initializer=glorot_uniform(seed=0)
)(x_data)

transfer_model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 1 * 10**-3
transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
num_epochs = 10
history_top = transfer_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=num_epochs,
)

base_model = transfer_model.layers[2]
base_model.trainable = True
fine_tune_at = 200
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

transfer_model.compile(
    optimizer=Adam(learning_rate=10**-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

history_base = transfer_model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=num_epochs * 2,
    # initial_epoch=history_top.epoch[-1],
)

Epoch 1/10
515/515 [==============================] - 42s 75ms/step - loss: 2.2380 - accuracy: 0.4448 - val_loss: 0.8896 - val_accuracy: 0.7308
Epoch 2/10
515/515 [==============================] - 38s 74ms/step - loss: 1.3120 - accuracy: 0.6286 - val_loss: 0.7496 - val_accuracy: 0.7699
Epoch 3/10
515/515 [==============================] - 38s 74ms/step - loss: 1.1261 - accuracy: 0.6806 - val_loss: 0.7451 - val_accuracy: 0.7685
Epoch 4/10
515/515 [==============================] - 38s 73ms/step - loss: 1.0123 - accuracy: 0.7048 - val_loss: 0.7292 - val_accuracy: 0.7809
Epoch 5/10
515/515 [==============================] - 38s 74ms/step - loss: 0.9214 - accuracy: 0.7293 - val_loss: 0.7244 - val_accuracy: 0.7823
Epoch 6/10
515/515 [==============================] - 39s 76ms/step - loss: 0.8704 - accuracy: 0.7432 - val_loss: 0.7348 - val_accuracy: 0.7782
Epoch 7/10
515/515 [==============================] - 38s 73ms/step - loss: 0.8126 - accuracy: 0.7546 - val_loss: 0.7305 - val_accuracy: